In [15]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""

from eos_database import *
from compressor_class import *
from compression import *
from gc_eos_soave import *
from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from viscosity import *
from plenum_system import *
import pdb

In [16]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)




In [17]:
n_x = 3
n_z = 11

def system_residuals(y, u0, plenum_sys):
    # Separa as variáveis
    x = y[:n_x]
    z = y[n_x:]
    
    # Substitua as expressões abaixo pelas suas equações do modelo em estado estacionário.
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x, z, u0)
    
    res_ode = np.array([ode_sym[i].item() for i in range(n_x)])
    
    # Calcula os resíduos das equações algébricas
    res_alg = np.array([alg_sym[i] for i in range(n_z)])

    res = np.concatenate((res_ode, res_alg))
    return res

def compute_steady_state(u0, plenum_sys, x0, z0):
    # Vetor inicial concatenado
    y0 = np.array(x0 + z0)
    
    # Chama o fsolve para encontrar os zeros da função de resíduos
    sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))
    
    # Separa a solução em x e z
    x_ss = sol[:n_x]
    z_ss = sol[n_x:]
    return x_ss, z_ss

if __name__ == '__main__':
    Phi, eta, Mach, Gimp, G2, Gdif, PHI, G2s, k = compression.character(compressor, m = 10, N = 750, Gi_1 = gas)
    x0 = [14.9919, 339.69, 0.42885]
    # E z0 pode ser (conforme seu código original):
    z0 = [6245.39, 6245.39, 321.672, 0.445562, 319.423, 0.503621, 320.097, 0.396345, 339.69, 0.42885, 0.514917]
    u0 = [4500, 300, 750, (10/0.38/(G2.P - 5000)**0.5), 5000]
    
    # Calcula os estados estacionários
    x_ss, z_ss = compute_steady_state(u0, plenum_sys, x0, z0)
    print("Estado estacionário (variáveis diferenciais):", x_ss)
    print("Estado estacionário (variáveis algébricas):", z_ss)

Estado estacionário (variáveis diferenciais): [ 14.99187014 339.69040519   0.42885043]
Estado estacionário (variáveis algébricas): [6.24539351e+03 6.24539351e+03 3.21672204e+02 4.45561887e-01
 3.19423402e+02 5.03620944e-01 3.20096967e+02 3.96345009e-01
 3.39690405e+02 4.28850426e-01 5.14917237e-01]


In [18]:
def test_evaluate_dae(plenum_sys, x_test, z_test, u0):# Parâmetros de entrada

    # Avaliação da função
    ode_values, alg_values = plenum_sys.evaluate_dae(None, x_ss, z_ss, u0)

    # Imprimindo os resultados
    print("Valores das ODEs:")
    for i, val in enumerate(ode_values):
        print(f"ODE {i}: {val}")

    print("\nValores das equações algébricas:")
    for i, val in enumerate(alg_values):
        print(f"Algebrica {i}: {val}")
    return ode_values, alg_values
ode_values, alg_values = test_evaluate_dae(plenum_sys, x_ss, z_ss, u0)

Valores das ODEs:
ODE 0: 0.0
ODE 1: 1.1667879349724045e-14
ODE 2: -0.0

Valores das equações algébricas:
Algebrica 0: -9.094947017729282e-13
Algebrica 1: 0.0
Algebrica 2: 1.2104207862521689e-15
Algebrica 3: 3.7765459686304125e-14
Algebrica 4: -1.6685314310489967e-15
Algebrica 5: 4.491839191910814e-14
Algebrica 6: 2.8990143619012088e-12
Algebrica 7: -1.4210854715202004e-14
Algebrica 8: 9.094947017729282e-13
Algebrica 9: 9.094947017729282e-13
Algebrica 10: 0.0


In [38]:
def run_simulation(x0, z0, plenum_sys):
    u0 = [4500, 300, 750, 0.65, 5000]
    # Definição dos símbolos
    x_sym = SX.sym('x', 3)
    z_sym = SX.sym('z', 11)
    u_sym = SX.sym('u', 5)

    # Avaliação das equações
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x_sym, z_sym, u_sym)

    # Funções para avaliação separada
    ode_fun = ca.Function('ode_fun', [x_sym, z_sym, u_sym], ode_sym)
    alg_fun = ca.Function('alg_fun', [x_sym, z_sym, u_sym], alg_sym)


    # Criação do DAE e integrador (mantido para referência)
    dae = {
        'x': x_sym,
        'z': z_sym,
        'p': u_sym,
        'ode': vertcat(*ode_sym),
        'alg': vertcat(*alg_sym)
    }
    
    integrator_solver = integrator('F', 'idas', dae, 0, 100)
    res = integrator_solver(x0=x0, z0=z0, p=u0)

    return res

run_simulation(x_ss,z_ss, plenum_sys)

{'adj_p': DM(0x0),
 'adj_u': DM(0x0),
 'adj_x0': DM(0x0),
 'adj_z0': DM(0x0),
 'qf': DM(0x1),
 'xf': DM([13.5185, 347.413, 0.341683]),
 'zf': DM([7995.44, 7995.44, 322.763, 0.43173, 321.106, 0.463829, 335.126, 0.325364, 347.413, 0.341683, 0.514917])}